# Hyperparametr-Tuning for Deep Neural network

## Importing Important Libraries

In [41]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import kerastuner

In [31]:
df = pd.read_csv('C:\\Users\\Sorcim\\tensorflow deep learning\\ANN hyperparameter-tuning\\Real_Combine.csv')

In [32]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [33]:
X = df.iloc[:, :-1] # independent featurs
y = df.iloc[:,-1] # dependent features

### Hyperparameters
* How many number of hidden layers we should have?
* how many number of neurons we should have in hidden layers?
* Learning rate

## Creating an ANN Model for regression problem

we are trying to run a loop on our model with different parameters and learning rates for picking up best parameters and learning rates

**How many hidden layers you should select? dun know? lets try hyperparameter tuning**

In [34]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                           min_value=32,
                                           max_value=512,
                                           step=32),
                              activation='relu'))
        model.add(layers.Dense(1, activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                     loss='mean_absolute_error',
                     metrics = ['mean_absolute_error'])
    return model

> for first step in for loop meaning that `i=1`, the `max_trials=5` and `executions_per_trial=3`, meaning that `5x3` which is equal to `15`.. 
so `15 iterations` will be there in this first step of the loop(e.g i=1) 15 times running...

>`val_mean_absolute_error` is use for regression problem.. this parameter will be change for classification problem.   



>`directory` name is project which means that all the weights will be stored in this project directory.

In [35]:
tuner = RandomSearch(build_model,
                    objective='val_mean_absolute_error',
                    max_trials=5,
                    executions_per_trial=3,
                    directory='project',
                    project_name='Ai Quality index')

In [36]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
tuner.search(X_train, y_train, epochs=50, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 17s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 01m 30s
INFO:tensorflow:Oracle triggered exit


In [39]:
tuner.results_summary()

Results summary
Results in project\Ai Quality index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 20
units_0: 384
learning_rate: 0.001
units_1: 288
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 15
units_0: 512
learning_rate: 0.001
units_1: 352
units_2: 256
units_3: 288
units_4: 480
units_5: 320
units_6: 192
units_7: 96
units_8: 224
units_9: 128
units_10: 160
units_11: 384
units_12: 224
units_13: 256
units_14: 64
units_15: 448
units_16: 256
units_17: 288
units_18: 352
units_19: 64
Score: nan
Trial summary
Hyperparameters:
num_layers: 19
units_0: 448
learning_rate: 0.01
units_1: 480
units_2: 448
units_3: 448
units_4: 128
units_5: 320
units_6: 352
units_7: 128
units_8: 320
units_9:

In [42]:
tuner = kerastuner.Hyperband(build_model,
                     objective='val_mean_absolute_error',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [43]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [45]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]



Trial 42 Complete [00h 00m 03s]
val_mean_absolute_error: 119.29370880126953

Best val_mean_absolute_error So Far: 51.55398178100586
Total elapsed time: 00h 02m 20s
INFO:tensorflow:Oracle triggered exit


In [47]:
tuner.results_summary()

Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 96
learning_rate: 0.01
units_1: 512
units_2: 160
units_3: 32
units_4: 64
units_5: 96
units_6: 320
units_7: 64
units_8: 224
units_9: 96
units_10: 128
units_11: 480
units_12: 288
units_13: 320
units_14: 224
units_15: 64
units_16: 192
units_17: 64
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 51.55398178100586
Trial summary
Hyperparameters:
num_layers: 9
units_0: 480
learning_rate: 0.01
units_1: 416
units_2: 64
units_3: 96
units_4: 288
units_5: 512
units_6: 384
units_7: 96
units_8: 128
units_9: 384
units_10: 32
units_11: 192
units_12: 288
units_13: 480
units_14: 96
units_15: 480
units_16: 448
units_17: 512
units_18: 128
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 57.41233444213867
Trial summary
Hyperparameters:
num_layers: 4
units_0: 480
learn